In [1]:
from audio_preprocessing.cut_audio_segment_from_audio_file import AudioProcessor
from feature_extraction.feature_extractor import FeatureExtractor
from feature_extraction.mfcc_images_extractor import MelSpectrogramImageExtractor
from data_processing.english_data_processing import ModelEvaluator
from itertools import combinations
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

In [2]:
def extract_mfcc_images(n_mfcc, figsize, input_folder, output_folder):
    mfccImageExtractor = MelSpectrogramImageExtractor(n_mfcc, figsize)
    mfccImageExtractor.extract_from_folders(input_folder, output_folder)

In [3]:
def cut_audio_segment_from_audio_file(frame_size, hop_size, threshold, input_folder, output_folder):
    segmentation = AudioProcessor(frame_size, hop_size, threshold)
    segmentation.process_folders(input_folder, output_folder)

In [4]:
def extract_feature(n_mfcc, input_folder, output_folder, selected_features):
    featureExtractor = FeatureExtractor(n_mfcc)
    featureExtractor.process_folder(input_folder, output_folder, selected_features)
    # featureExtractor.process_folder(input_folder, output_folder, ['mfcc','mfcc_statistics', 'zcr', 'pitch', 'rms'])

In [5]:
def model(featured_data):
    models = [RandomForestClassifier, ExtraTreesClassifier]
    
    df_train_test = pd.read_csv(featured_data)
    df_shuffled = df_train_test.sample(frac=1, random_state=42).reset_index(drop=True)

    modelEvaluator = ModelEvaluator(models)
    best_model, best_scaler, best_avg_acc = modelEvaluator.evaluate_models(df_shuffled)
    return best_model, best_scaler, best_avg_acc

In [6]:
def process_data(frame_size, hop_size, threshold, raw_folder, cutted_folder, n_mfcc_list, featured_data_path, n_mfcc_for_images, images_folder, features):
    n = len(features)

    results = []

    for n_mfcc in n_mfcc_list:
        for r in range(0, n):
            for combo in combinations(range(n), r):
                selected_features = [features[i] for i in range(n) if i not in combo]
                extract_feature(n_mfcc, cutted_folder, featured_data_path, selected_features)
                current_model, current_scaler, current_avg_acc = model(featured_data_path)
                print(f"Scaler: {current_scaler}, Model: {current_model}, n_mfcc: {n_mfcc}, Selected features: {selected_features}, Accuracy: {current_avg_acc}")
                results.append({'Scaler': current_scaler, 'Model': current_model, 'N_MFCC': n_mfcc, 'Selected Features': selected_features, 'Average Accuracy': current_avg_acc})

    results_df = pd.DataFrame(results)
    ranked_results = results_df.sort_values(by='Average Accuracy', ascending=False)

    return ranked_results

In [7]:
ranked_results = process_data(256, 128, 0.0005, 'D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/combined_data', 'D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/cleaned_data', [26,40], 'D:/data_analysis/speech_emotion_recognition/notebooks/feature0.csv', 100, 'D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/images', ['mfcc','mfcc_statistics', 'zcr', 'pitch', 'rms'])

2024-05-23 03:52:37.422 | INFO     | feature_extraction.feature_extractor:process_folder:148 - Processing input folder: D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/cleaned_data
2024-05-23 03:55:05.322 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature0.csv
2024-05-23 03:55:05.487 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....
2024-05-23 03:56:34.659 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: None, accuracy: 0.6278300718355214
2024-05-23 03:56:52.054 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> - scaler: None, accuracy: 0.6244707290892577
2024-05-23 03:58:19.613 | INFO     | data_processing.english_da

Scaler: MaxAbsScaler, Model: <class 'sklearn.ensemble._forest.RandomForestClassifier'>, n_mfcc: 26, Selected features: ['mfcc', 'mfcc_statistics', 'zcr', 'pitch', 'rms'], Accuracy: 0.6306498224754357


2024-05-23 04:07:36.411 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature0.csv
2024-05-23 04:07:36.542 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....
2024-05-23 04:08:57.360 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: None, accuracy: 0.6246504830319545
2024-05-23 04:09:14.417 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> - scaler: None, accuracy: 0.6226506481710842
2024-05-23 04:10:36.975 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: StandardScaler(), accuracy: 0.625195359590455
2024-05-23 04:10:53.476 | INFO     | data_

Scaler: MaxAbsScaler, Model: <class 'sklearn.ensemble._forest.RandomForestClassifier'>, n_mfcc: 26, Selected features: ['mfcc_statistics', 'zcr', 'pitch', 'rms'], Accuracy: 0.6295577574106185


2024-05-23 04:18:36.004 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature0.csv
2024-05-23 04:18:36.049 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....
2024-05-23 04:19:21.026 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: None, accuracy: 0.6012925439682932
2024-05-23 04:19:32.108 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> - scaler: None, accuracy: 0.6031999009165221
2024-05-23 04:20:16.886 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: StandardScaler(), accuracy: 0.6012019651556436
2024-05-23 04:20:28.000 | INFO     | data

...

2024-05-23 05:06:20.025 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature0.csv
2024-05-23 05:06:20.163 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....
2024-05-23 05:07:42.414 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: None, accuracy: 0.6200142845347205
2024-05-23 05:07:59.306 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> - scaler: None, accuracy: 0.6213775906200975
2024-05-23 05:09:21.439 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: StandardScaler(), accuracy: 0.6250144496738502
2024-05-23 05:09:37.954 | INFO     | data

Scaler: MinMaxScaler, Model: <class 'sklearn.ensemble._forest.RandomForestClassifier'>, n_mfcc: 26, Selected features: ['mfcc_statistics', 'pitch', 'rms'], Accuracy: 0.6269230451655521


2024-05-23 05:17:45.903 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature0.csv
2024-05-23 05:17:46.068 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....
2024-05-23 05:19:13.778 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: None, accuracy: 0.6196513912971678
2024-05-23 05:19:30.290 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> - scaler: None, accuracy: 0.623558583106267
2024-05-23 05:20:52.387 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: StandardScaler(), accuracy: 0.6231967632730575
2024-05-23 05:21:09.227 | INFO     | data_

Scaler: RobustScaler, Model: <class 'sklearn.ensemble._forest.RandomForestClassifier'>, n_mfcc: 26, Selected features: ['mfcc_statistics', 'zcr', 'rms'], Accuracy: 0.625558087688878


2024-05-23 05:34:32.362 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature0.csv
2024-05-23 05:34:33.002 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....


In [ ]:
print(ranked_results)